## 1. Классические Американские имена

<p>Как изменились американские имена с 1920 года? Какие имена остаются популярными уже более 100 лет и как эти имена соотносятся с более поздними популярными именами?  </p>
<p>Мы будем работать с данными, предоставленными Управлением социального обеспечения США, в котором указаны имена, а также количество и пол детей, которым они давались каждый год. В целях скорости обработки я ограничила данные именами, которые были даны более чем 5000 американским младенцам в данном году. Данные охватывают 101 год, с 1920 по 2020 год.</p>

<h3 id="baby_names"><code>baby_names</code></h3>
<table>
<thead>
<tr>
<th style="text-align:left;">column</th>
<th>type</th>
<th>значение</th>
</tr>
</thead>
<tbody>
<tr>
<td style="text-align:left;"><code>year</code></td>
<td>int</td>
<td>год</td>
</tr>
<tr>
<td style="text-align:left;"><code>first_name</code></td>
<td>varchar</td>
<td>Имя</td>
</tr>
<tr>
<td style="text-align:left;"><code>sex</code></td>
<td>varchar</td>
<td><code>пол</code> ребенка </td>
</tr>
<tr>
<td style="text-align:left;"><code>num</code></td>
<td>int</td>
<td>количество детей данного <code>пола</code>  в этот  <code>год</code></td>
</tr>
</tbody>
</table>
<p>Начнем с поиска имен, которые остались популярными сквозь года.</p>

In [51]:
%%sql
postgresql:///names
    
-- Выберем имена и общее количество детей с таким именем.
select first_name, sum(num)
from baby_names
-- Сгруппируем по имени  
group by first_name
--Отфильтруем имена так, чтобы отображались
-- имена,которые присутствуют в каждом из 101 года.
having count(distinct year)=101
-- Упорядочим по общему количеству детей с таким именем, по убыванию
order by sum(num) desc


8 rows affected.


first_name,sum
James,4748138
John,4510721
William,3614424
David,3571498
Joseph,2361382
Thomas,2166802
Charles,2112352
Elizabeth,1436286


## 2. Вне времени или модно?
<p> Похоже, существует множество вечных традиционно мужских имен </p>
<p> Теперь давайте расширим наше понимание набора данных, рассмотрев все имена. Мы попытаемся определить тип популярности, которой пользовалось каждое имя в наборе данных. Было ли это имя классическим и популярным на протяжении многих лет или просто модным, популярным лишь несколько лет? Давай выясним. </p>

In [ ]:
%%sql

-- Выберем first_name, и sum детей, которым когда-либо было дано это имя, 
-- и popularity_type
select first_name, sum(num),
-- Классифицируем имена таким образом 'Classic', 
--'Semi-classic', 'Semi-trendy', или 'Trendy'
-- в зависимости от того, как часто имя было дано ребенку
-- Назовем этот столбец popularity_type
    Case when count(first_name)>80 then 'Classic'
        when count(first_name)>50 then 'Semi-classic'
        when count(first_name)>20  then 'Semi-trendy'
        else 'Trendy' end as popularity_type
from baby_names 
group by first_name
-- Упорядочим результаты в алфавитном порядке имен, first_name
order by first_name

 * postgresql:///names
547 rows affected.


## 3. Самые популярные женские имена с 1920 года.

<p>Поскольку при поиске классических американских имен в первой задаче мы не получили много традиционно женских имен, давайте ограничим наш поиск именами, которые давали младенцам женского пола. </p>
<p>Мы можем использовать эту возможность, чтобы попрактиковаться в оконных функциях, присваивая ранг женским именам на основе количества детей, которым когда-либо давали это имя. Какие женские имена самые популярные с 1920 года?</p>

In [ ]:
%%sql

-- Создадим ранг имен, основываясь на sum детей, которым когда-либо давали
-- это имя (по убыванию), назовем столбец name_rank
-- Также выберем name_rank, first_name, и sum детей, 
-- которым когда-либо было дано это имя
SELECT
    RANK() OVER(ORDER BY SUM(num) DESC) AS name_rank,
    first_name, SUM(num)
FROM baby_names
-- Отфильтруем так, чтобы отображались только женские имена
WHERE sex = 'F'
GROUP BY first_name
-- Лимитируем результат до 10
LIMIT 10;

## 4. Выбор имени для ребенка
<p> Давайте представим ситуацию, кто-то услышав об этом проекте, попросил помощи с выбором имени для ребенка, девочки. Родителю не нравится ни одно из самых популярных имен, которые мы нашли в предыдущем задании. </p>
<p>Этот человек хочет выбрать традиционно женское имя, оканчивающееся на букву «а», так как слышал, что гласные в детских именах сейчас в моде. Он также ищет имя, которое было популярно с 2015 года. </p>
<p>Давайте посмотрим, что мы можем сделать, чтобы найти варианты для этого человека</p>

In [ ]:
%%sql

-- выберем имена
select first_name
from baby_names
-- Отфильтруем результаты на те, где пол указан как "F", а год больше 2015, 
-- и имя заканчивается на "а"
where sex='F' and year>2015 and first_name like '%a'
-- Сгруппируем по имени и упорядочиваем по общему количеству детей с этим именем
group by first_name
order by sum(num) desc

## 5. Когда это имя стало популярным?
<p>По результатам предыдущего задания мы видим, что Оливия — самое популярное женское имя, оканчивающееся на «А» с 2015 года. Когда имя Оливия стало таким популярным?</p>
<p>Давайте исследуем появление имени Оливия в тренде с помощью оконной функции.</p>

In [ ]:
%%sql

-- Выберем year, first_name, количество Оливии в этом году, 
-- и совокупное количество Оливии
-- Посчитаем общее количество детей, которых назвали Оливия до 
-- этого года и назовем столбец cumulative_olivias
select year, first_name, num, 
    sum(num) over(order by year) as cumulative_olivias
from baby_names
-- Отфильтруем, чтобы результаты высвечивались только для имени Оливия
where first_name='Olivia'
-- Упорядочим по годам, от самого раннего до самого последнего года
order by year asc


## 6. Одинаковые имена для мужчин
<p> Имя Оливия стало популярным в 1991 году. Давайте теперь рассмотрим традиционно мужские имена. В первом задании мы увидели, что в нашем 101-летнем наборе данных каждый год по меньшей мере 5000 младенцев даются девять традиционно мужских имен! Эти имена являются классикой, но появление в наборе данных каждый год не обязательно означает, что вечные имена были самыми популярными. Давайте подробнее рассмотрим популярные мужские имена.</p>
<p>В следующих двух задачах мы доведем список каждого года до самого популярного мужского имени в этом году. Перед нами задача: найти мужское имя, данное наибольшему числу младенцев в год. </p>
<p>В SQL одним из подходов является использование подзапроса. Сначала мы можем написать запрос, который выбирает <code>year</code> и максимальное <code>num</code> детей, получивших  мужское имя в этом году. В следующей задаче мы можем использовать код из этой задачи в качестве подзапроса. </p>

In [ ]:
%%sql

-- Выберем год и максимальное кол-во детей, получивших это мужское имя в этом году, 
-- назовем это max_num
select year, max(num) as max_num
from baby_names
-- Отфильтруем результат так, чтобы исключить женские имена
where sex='M'
group by year;

## 7. Самые популярные мужские имена за последние годы
<p>В предыдущем задании мы нашли максимальное количество младенцев, получивших одно мужское имя, в каждом году. Интересно, что самое популярное имя каждый год менялось от менее чем 20 000 детей до более чем 90 000 </p>
<p>В этой задаче мы узнаем, какое самое популярное мужское имя для каждого года в нашем наборе данных. </p>

In [ ]:
%%sql

-- Выберем год, имя, присвоенное наибольшему количеству младенцев мужского пола,
-- и количество детей, получивших это имя.
select b.year, b.first_name, b.num
from baby_names as b
-- Присоединяем baby_names к коду последней задачи в виде подзапроса
Inner join (select year, max(num) as max_num
            from baby_names
            where sex='M'
            group by year) as sub
on sub.year= b.year and sub.max_num=b.num
-- Упорядочим результаты по годам по убыванию
order by year desc

## 8. Какие имена наибольшее количество лет на первом месте?
<p> Ной и Лиам лидировали в последние несколько лет, но если мы прокрутим результаты вниз, окажется, что Майкл и Джейкоб также провели немало лет в качестве лидеров. Какое имя было номером один на протяжении наибольшего количества лет? Давайте воспользуемся обычным табличным выражением, чтобы это выяснить. </p>

In [ ]:
%%sql


-- Используем код из предыдущего задания в качестве common table expression
with smth as (select b.year, b.first_name, b.num
from baby_names as b
Inner join (select year, max(num) as max_num
            from baby_names
            where sex='M'
            group by year) as sub
on sub.year= b.year and sub.max_num=b.num
order by year desc)

-- Выберем first_name и количество лет, когда это имя было топ 1; 
-- назовем это count_top_name
select first_name,
    count(first_name) as count_top_name
from smth
-- Сгруппируем по first_name и упорядочим по count_top_name по убыванию
group by first_name
order by count_top_name desc

## 8. Результат
<p> Самым популярным мужским именем оказалось имя Майкл, а самое популярное женское имя- это Оливия </p>